In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.3.0 --quiet
%pip install datasets
# Installing the Reinforcement Learning library directly from github.
%pip install git+https://github.com/lvwerra/trl.git@25fa1bd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.8 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: Ignored the following yanked versions: 0.3.0a0
ERROR: Could not find a version that satisfies the requirement torchdata==0.5.1 (from versions: 0.3.0a1, 0.3.0, 0.6.0, 0.6.1, 0.7.0, 0.7.1, 0.8.0, 0.9.0, 0.10.0, 0.10.1, 0.11.0)
ERROR: No matching distribution found for torchdata==0.5.1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 183.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 176.5 

In [4]:
pip install -U datasets

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [datasets]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
from datasets import load_dataset
from transformers import pipeline
from trl.core import LengthSampler
import torch
from tqdm import tqdm


In [ ]:
def build_dataset(model_name, dataset_name, min_len, max_len):
    dataset = load_dataset(dataset_name, split="train")
    dataset = dataset.filter(lambda x: min_len < len(x["dialogue"]) <= max_len, batched=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize(sample):
        prompt = f"Summarize the following conversation.\n\n{sample['dialogue']}\n\nSummary:\n"
        sample["input_ids"] = tokenizer.encode(prompt, truncation=True, max_length=512) #creates token IDs for words
        sample["query"] = tokenizer.decode(sample["input_ids"]) #revert that back to original prompt text
        return sample

    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")
    return dataset.train_test_split(test_size=0.2)

model_name = "google/flan-t5-base"
dataset = build_dataset(model_name, "knkarthick/dialogsum", 200, 1000)

print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 8017
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2005
    })
})


In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
peft_model = get_peft_model(base_model, lora_config)



config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:


ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model, is_trainable=True, torch_dtype=torch.bfloat16)

ref_model = create_reference_model(ppo_model)



In [ ]:


reward_model_raw = "facebook/roberta-hate-speech-dynabench-r4-target"
reward_tokenizer = AutoTokenizer.from_pretrained(reward_model_raw, device_map="auto")
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_raw, device_map="auto")


reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model,
    tokenizer=reward_model,
    device=0 if torch.cuda.is_available() else -1,
    framework="pt"  
)

reward_kwargs = {
    "top_k": None,
    "function_to_apply": "none",
    "batch_size": 16
}

not_hate_index = 0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
def collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

ppo_config = PPOConfig(
    model_name=model_name,
    learning_rate=1.41e-5,
    ppo_epochs=1,
    mini_batch_size=4,
    batch_size=16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
ppo_trainer = PPOTrainer(config=ppo_config, model=ppo_model, ref_model=ref_model,
                         tokenizer=tokenizer, dataset=dataset["train"], data_collator=collator)


In [ ]:
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reward_model.to(device)

toxicity_input = reward_tokenizer(non_toxic_text, return_tensors="pt").to(device)

logits = reward_model(**toxicity_input).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (high): {nothate_reward}')


toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reward_model.to(device)
toxicity_input = reward_tokenizer(toxic_text, return_tensors="pt").to(device)
logits = reward_model(**toxicity_input).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')
not_hate_index = 0  # usually class 0 = "not hate"
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (low): {nothate_reward}')


# Higher reward = less toxic text
# Lower reward = more toxic text

logits [not hate, hate]: [3.114102363586426, -2.489619016647339]
probabilities [not hate, hate]: [0.9963293671607971, 0.0036706042010337114]
reward (high): [3.114102363586426]
logits [not hate, hate]: [-0.6921166181564331, 0.3722708821296692]
probabilities [not hate, hate]: [0.2564719021320343, 0.7435281276702881]
reward (low): [-0.6921166181564331]


In [ ]:
from transformers import pipeline
import torch
from trl.core import LengthSampler
from tqdm import tqdm

device = 0 if torch.cuda.is_available() else -1

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model_raw,
    device=device,
    framework="pt"
)

reward_logits_kwargs = {
    "top_k": None,
    "function_to_apply": "none",  
    "batch_size": 16
}
reward_probabilities_kwargs = {
    "top_k": None,
    "function_to_apply": "softmax",  
    "batch_size": 16
}

print("Reward model output:")
print("For non-toxic text:")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))

print("For toxic text:")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

max_ppo_steps = 20
not_hate_index = 0  

for step, batch in tqdm(enumerate(ppo_trainer.dataloader), total=max_ppo_steps):
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()
        generation_kwargs["max_new_tokens"] = max_new_tokens

        output = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        summary_tensors.append(output.squeeze()[-max_new_tokens:])

    batch["response"] = [tokenizer.decode(r, skip_special_tokens=True) for r in summary_tensors]

    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_logits_kwargs)
    reward_tensors = [torch.tensor(r[not_hate_index]["score"]) for r in rewards]

    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f"Step {step + 1}")
    print(f"  KL Divergence        : {stats['objective/kl']}")
    print(f"  Mean Return (Reward) : {stats['ppo/returns/mean']}")
    print(f"  Advantage Mean       : {stats['ppo/policy/advantages_mean']}")
    print("-" * 50)


Reward model output:
For non-toxic text:
[{'label': 'nothate', 'score': 3.114102363586426}, {'label': 'hate', 'score': -2.489619016647339}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.0036706042010337114}]
For toxic text:
[{'label': 'hate', 'score': 0.3722708821296692}, {'label': 'nothate', 'score': -0.6921166181564331}]
[{'label': 'hate', 'score': 0.7435281276702881}, {'label': 'nothate', 'score': 0.2564719021320343}]


  5%|▌         | 1/20 [00:18<05:50, 18.43s/it]

Step 1
  KL Divergence        : 0.0029426664113998413
  Mean Return (Reward) : 1.6667838096618652
  Advantage Mean       : -2.2025901103006618e-08
--------------------------------------------------



 10%|█         | 2/20 [00:33<04:54, 16.37s/it]

Step 2
  KL Divergence        : 0.0022393250837922096
  Mean Return (Reward) : 1.9268946647644043
  Advantage Mean       : 1.3280663679893223e-08
--------------------------------------------------


...

Step 18
  KL Divergence        : 0.18081429600715637
  Mean Return (Reward) : 1.954123616218567
  Advantage Mean       : -3.4959867889483576e-08
--------------------------------------------------



 95%|█████████▌| 19/20 [04:56<00:15, 15.59s/it]

Step 19
  KL Divergence        : 0.19736091792583466
  Mean Return (Reward) : 2.006293773651123
  Advantage Mean       : 1.207381075118974e-07
--------------------------------------------------


100%|██████████| 20/20 [05:09<00:00, 15.49s/it]

Step 20
  KL Divergence        : 0.05986378714442253
  Mean Return (Reward) : 2.3082714080810547
  Advantage Mean       : -3.8824705228535095e-08
--------------------------------------------------


In [ ]:
import os

save_path = "/content/drive/MyDrive/ppo_tuned_model_FLAN"

os.makedirs(save_path, exist_ok=True)

ppo_trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/ppo_tuned_model_FLAN/tokenizer_config.json',
 '/content/drive/MyDrive/ppo_tuned_model_FLAN/special_tokens_map.json',
 '/content/drive/MyDrive/ppo_tuned_model_FLAN/spiece.model',
 '/content/drive/MyDrive/ppo_tuned_model_FLAN/added_tokens.json',
 '/content/drive/MyDrive/ppo_tuned_model_FLAN/tokenizer.json')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ref_model.to(device)
ppo_model.to(device)

batch_size = 25
compare_results = {}

df_batch = dataset["test"][0:batch_size]
compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    input_tensor = torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device)

    with torch.no_grad():
        ref_output = ref_model.generate(input_ids=input_tensor, **generation_kwargs)
        summary_tensors_ref.append(ref_output.squeeze()[-gen_len:])

        ppo_output = ppo_model.generate(input_ids=input_tensor, **generation_kwargs)
        summary_tensors.append(ppo_output.squeeze()[-gen_len:])

compare_results["response_before"] = [tokenizer.decode(tensor, skip_special_tokens=True) for tensor in summary_tensors_ref]
compare_results["response_after"] = [tokenizer.decode(tensor, skip_special_tokens=True) for tensor in summary_tensors]

texts_before = [q + " " + r for q, r in zip(compare_results["query"], compare_results["response_before"])]
texts_after = [q + " " + r for q, r in zip(compare_results["query"], compare_results["response_after"])]

reward_kwargs = {
    "top_k": None,
    "function_to_apply": "none",
    "batch_size": 16
}

rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)

not_hate_index = 0  

compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

import pandas as pd
df_compare = pd.DataFrame(compare_results)
pd.set_option("display.max_colwidth", 200)
print(df_compare[["query", "response_before", "reward_before", "response_after", "reward_after"]])

100%|██████████| 25/25 [00:40<00:00,  1.64s/it]
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


                                                                                                                                                                                                      query  \
0   Summarize the following conversation. #Person1#: Hello, Jane. #Person2#: Hi, Harry. Did you have a good summer holiday? #Person1#: Sure. I went for my holiday on my uncle's farm. #Person2#: Really...   
1   Summarize the following conversation. #Person1#: Hello, Jill. #Person2#: Tom, You're back, come in please. How are you? #Person1#: Fine, only a little tired. #Person2#: You'll recover after a good...   
2   Summarize the following conversation. #Person1#: Hello? Is that Mark? #Person2#: How are you? I haven't heard from you in ages. #Person1#: I've been overseas, So have you been busy lately? #Person...   
3   Summarize the following conversation. #Person1#: Hi Paul. How are you, friend. #Person2#: Not good. My cousin is driving me up the wall. #Person1#: How so? #Person2#: H

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 500)

df_compare_results = pd.DataFrame(compare_results)

df_compare_results["reward_diff"] = df_compare_results["reward_after"] - df_compare_results["reward_before"]

df_compare_results_sorted = df_compare_results.sort_values(by="reward_diff", ascending=False).reset_index(drop=True)

df_compare_results_sorted[[
    "query",
    "response_before", "reward_before",
    "response_after", "reward_after",
    "reward_diff"
]]


,query,response_before,reward_before,response_after,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: You only have an hour for lunch? #Person2#: No, now I only have 45 minutes. #Person1#: That's not enough. Where are we going? #Person2#: We can go to a place near the mall. #Person1#: Oh, alright, let's go across the street. We can eat at Tony's Italian restaurant. I love their pizza. #Person2#: I love their food, too. But they are really slow. Last week I waited 30 minutes for my food. #Person1#: OK. Let's have sushi at Dave's. We can be in a...",Person1 has to travel 45 minutes for lunch. She will have extra time chances for scarfing down dinner. Also she will have sushi after school.,1.386092,The meeting starts at 12:00 today and before the clock strikes 12:30 pm.,3.388463,2.002372
1,"Summarize the following conversation. #Person1#: There will be a party at my new house this Saturday. Would you like to come? #Person2#: That sounds good, but I have French class in the morning and dance class in the afternoon. #Person1#: That's OK. The party is to start in the evening, and you can come after the dance class. #Person2#: Great! Should I bring something? #Person1#: Yes, it's a potluck party, so you should prepare something to eat. #Person2#: No problem. A roast turkey, salad, ...",The party is at Person1's house on Saturday with cake and attitude.,2.791882,"The party starts in evening at Person1's New House this Saturday. People are asking about some things to bring to the party that include apples, bananas, applesauce, and powdered sugar.",3.583677,0.791795
2,"Summarize the following conversation. #Person1#: Hello, Jane. #Person2#: Hi, Harry. Did you have a good summer holiday? #Person1#: Sure. I went for my holiday on my uncle's farm. #Person2#: Really? What interesting things did you do there? #Person1#: I helped get in some rice, take care of the fruit garden and drive the tractor. #Person2#: Drive a tractor? #Person1#: Yes. It was easy to learn. Did you go away for your holiday, Jane? #Person2#: Oh, no. I just stayed at home. My mother has bee...","Person1: Hi, Harry. Note that the last person is Jane Hyde when she is off from work.",2.459215,"Jane went to her uncle's farm three times, driving the tractor, taking care of the fruit garden, and teaching everyone. She also taught herself how to drive a lawn mower on the farm.",3.215431,0.756216
3,"Summarize the following conversation. #Person1#: Here we are, Ryan! This is where we're going to celebrate! #Person2#: It's a ETV palace! I'm glad I brought my platinum card. #Person1#: You won't need it. Stanley, my best man, is going to treat everybody! #Person2#: Where is Stanley? It was his idea to have the bachelor's party at a ETV, wasn't it? #Person1#: If it were up to Stanley, we'd have the wedding in the ETV! He loves to sing. #Person2#: Then I bet he's really good! #Person1#: Well,...","One of Ryan's younger friends and family wants to celebrate honeymoon to parents who live close to it. They're going to hold their wedding in the ETV, which is new. The parents want to hold it in the same spot as People.",2.921001,The entire family showed up on Time Night Thursday evening and were really excited for Ron and Ryan's bachelor's party tonight. They think Stanley is a really good singer. They'll move towards the ETV.,3.626835,0.705834
4,"Summarize the following conversation. #Person1#: What do you like to do in your spare time? #Person2#: I like playing chess. #Person1#: Do you have any hobbies besides playing chess? #Person2#: I'm afraid not. #Person1#: Do you have any hobbies like playing tennis or things like that? #Person2#: Oh, yes. I like playing basketball. #Person1#: Can you tell me why you like it? #Person2#: Because I like the feeling of cooperating with others. Summary: </s>","Person1: If you like basketball, you're more of a hopscotch person.",1.903953,#Person1#: I am a friend of Person2 so I need to know something about you and for me.,2.56